In [38]:
# See the result in: http://127.0.0.1:8050/

import pandas as pd
import altair as alt
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_vega_components as dvc

alt.data_transformers.enable("vegafusion")

# Load data
df = pd.read_csv("/Users/nayemontiel18/Library/CloudStorage/OneDrive-UBC/UBCO/MDS/BLOCK5/Data551/Dashboard/DATA551_FireAnalysis/data/processed/outputN.csv", low_memory=False)

## -- Starting Plots Definition --

## -- Plot 1: "altair-chart-1" --

#alt.themes.enable('dark')

wildfires_causes = df.groupby(['FIRE_YEAR', 'state_descriptions', 'FIRE_SIZE_CLASS', 'STAT_CAUSE_DESCR']).size().reset_index(name='COUNT')

# Function to create Altair chart
def create_altair_chart(data):
    title = alt.TitleParams(
    text='Understanding Wildfires: Causes and Estimated Fire Size Classes',
    subtitle='The fire size classes range from A (small) to G (large)')
    
    chart = alt.Chart(data, title=title).mark_square().encode(
        alt.X('FIRE_SIZE_CLASS:N', title=None, axis=alt.Axis(labelAngle=0, ticks=False)),
        alt.Y('STAT_CAUSE_DESCR:N', title=None, sort='color', axis=alt.Axis(ticks=False, labels=True)), 
        alt.Color('count(FIRE_SIZE_CLASS)', title= 'Number of wildfires', scale=alt.Scale(scheme='darkred')), 
        size='count(FIRE_SIZE_CLASS)',
        tooltip=[
        alt.Tooltip('FIRE_SIZE_CLASS:N', title='Fire Size Class'),
        alt.Tooltip('STAT_CAUSE_DESCR:N', title='Cause of Fire'),
        alt.Tooltip('count(FIRE_SIZE_CLASS)', title='Number of Fires')
        ]
    ).properties(width=300, height=300).configure_view(stroke='transparent')
    return chart.to_dict(format="vega")

## -- Plot 2: "altair-chart-2" --

#Number of fires 
causes_grouped = df[df['CAUSES'].isin(['Human', 'Lightning'])].groupby(['FIRE_YEAR', 'state_descriptions', 'geographic_areas_desc', 'CAUSES']).size().reset_index(name='COUNT')

def create_altair_chart2(data):
    title2 = alt.TitleParams(
        text='Proportion of Fires',
        subtitle='By Geographic Area and Cause')
    
    chart2 = alt.Chart(data, title=title2).mark_bar().encode(
        alt.X('sum(COUNT)', stack='normalize', title=''),
        alt.Y('geographic_areas_desc:O', sort='y', title=""),
        alt.Color('CAUSES', title="", scale=alt.Scale(scheme='darkred'), legend=alt.Legend(orient='none',
            legendX=90, legendY=-20,
            direction='horizontal',
            titleAnchor='middle')),
        tooltip=[
            alt.Tooltip('sum(COUNT)', title='Fires', format=',d'),
            alt.Tooltip('geographic_areas_desc:O', title='Area'),
            alt.Tooltip('CAUSES', title='Cause')
        ]
    ).properties(width=300, height=300)
    return chart2.to_dict(format="vega")

## -- Plot 3: "altair-chart-3" --

# Group by FIRE_YEAR, STATE, and MONTH, calculate the count, and reset the index
grouped_df = df[df['CAUSES'].isin(['Human', 'Lightning'])].groupby(['FIRE_YEAR', 'state_descriptions', 'CAUSES', 'MONTH']).size().reset_index(name='COUNT')

# Define the sort order
sort_order = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
# Sorting dataframe by the desired order
grouped_df['MONTH'] = pd.Categorical(grouped_df['MONTH'], categories=sort_order, ordered=True)
grouped_df = grouped_df.sort_values(by='MONTH')


def create_altair_chart3(data):
    title3 = alt.TitleParams(
                text='Average number of fires',
                subtitle='By Month and Cause')

    chart3 = (alt.Chart(data, title=title3).mark_line().encode(
    alt.X('MONTH:O', title=None, sort=sort_order, axis=alt.Axis(labelAngle=0)),
    alt.Y('average(COUNT)', title=None),
    alt.Color('CAUSES:O', title= '', scale=alt.Scale(scheme='darkred'), legend=alt.Legend(orient='none',
    legendX=120, legendY=-20, direction='horizontal', titleAnchor='middle'))
    ).properties(width=355, height=270,
    ) + alt.Chart(data).mark_point().encode(
    alt.X('MONTH:O', title=None, sort=sort_order),
    alt.Y('average(COUNT)'),
    alt.Color('CAUSES:O', title='', scale=alt.Scale(scheme='darkred')),
    tooltip=[
    alt.Tooltip('MONTH:O', title='Month'),
    alt.Tooltip('average(COUNT):Q', title='Count', format='.2f'),
    alt.Tooltip('CAUSES:O', title='Cause')]
    ))
    return chart3.to_dict(format="vega")

## -- Plot 4: "altair-chart-4" --

# Total acres burned
causes_grouped2 = df[df['CAUSES'].isin(['Human', 'Lightning'])].groupby(['FIRE_YEAR', 'state_descriptions', 'geographic_areas_desc', 'CAUSES'])['FIRE_SIZE'].sum().reset_index(name='SUM')

def create_altair_chart4(data):
    title4 = alt.TitleParams(
        text='Proportion of Acres Burned',
        subtitle='By Geographic Area and Cause')
    
    chart4 = alt.Chart(data, title=title4).mark_bar().encode(
        alt.X('sum(SUM)', stack='normalize', title=''),
        alt.Y('geographic_areas_desc:O', sort='y', title=""),
        alt.Color('CAUSES', title="", scale=alt.Scale(scheme='darkred'), legend=alt.Legend(orient='none',
            legendX=90, legendY=-20,
            direction='horizontal',
            titleAnchor='middle')),
        tooltip=[
            alt.Tooltip('sum(SUM)', title='Acres', format=',d'),
            alt.Tooltip('geographic_areas_desc:O', title='Area'),
            alt.Tooltip('CAUSES', title='Cause')
        ]
    ).properties(width=300, height=300)
    return chart4.to_dict(format="vega")

## -- End of plots --

# Initialize Dash app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Options for filters (New version)
states = sorted([{'label': state, 'value': state} for state in df['state_descriptions'].unique()], key=lambda x: x['label'])
default_state = 'Alaska'

# Define layout
app.layout = html.Div([
    # Main container for the entire layout
    html.Div([
        # Filters sidebar (dbc.Col with width=3)
        dbc.Col([
            html.Label('Fire Year', className="filter-label"),
            dcc.RangeSlider(
                id='year-slider',
                min=df['FIRE_YEAR'].min(),
                max=df['FIRE_YEAR'].max(),
                step=1,
                value=[df['FIRE_YEAR'].min(), df['FIRE_YEAR'].max()],
                marks={df['FIRE_YEAR'].min(): str(df['FIRE_YEAR'].min()),
                       df['FIRE_YEAR'].max(): str(df['FIRE_YEAR'].max())}
            ),
            html.Div(id='slider-output-container-mi', className="slider-output"),
            html.Label('State', className="filter-label"),
            dcc.Dropdown(
                id='state-dropdown',
                options=states,
                multi=True,
                value=[default_state],
                className="shadow-style"
            ),
            html.Button("SELECT ALL", id="select-all", n_clicks=0, style={'margin-top': '10px'}),
            html.Br()
        ], width=3),  # End of Filters sidebar

        # First column of plots (dbc.Col with width=6)
        dbc.Col([
            html.Br(),
            dvc.Vega(
                id="altair-chart-1",
                opt={"renderer": "svg", "actions": False},
                spec=create_altair_chart(wildfires_causes),
            ),
            html.Br(),
            dvc.Vega(
                id="altair-chart-2",
                opt={"renderer": "svg", "actions": False},
                spec=create_altair_chart2(causes_grouped),
            ),
        ], width=3.5),  # End of first column

        # Second column of plots (dbc.Col with width=6)
        dbc.Col([
            html.Br(),
            dvc.Vega(
                id="altair-chart-3",
                opt={"renderer": "svg", "actions": False},
                spec=create_altair_chart3(grouped_df),
            ),
            html.Br(),
            dvc.Vega(
                id="altair-chart-4",
                opt={"renderer": "svg", "actions": False},
                spec=create_altair_chart4(causes_grouped2),
            ),
        ], width=4)  # End of second column
    ], className="Container")  # End of Main container
])

# Update Altair chart based on filter selection
from dash.exceptions import PreventUpdate

@app.callback(
    Output("state-dropdown", "value"),
    [Input("select-all", "n_clicks")]
)
def select_all_states(n_clicks):
    if n_clicks is None:
        raise PreventUpdate
    return all_states.tolist()
    
@app.callback(
    [Output('altair-chart-1', 'spec'),
     Output('altair-chart-2', 'spec'),
     Output('altair-chart-3', 'spec'),
     Output('altair-chart-4', 'spec')],
    [Input('year-slider', 'value'),
     Input('state-dropdown', 'value')]
)
def update_altair_chart(year_range, selected_states):
    
    if not selected_states:
        # If no states are selected, return None for all charts
        return [None] * 4

    dff = wildfires_causes.copy()
    dff2 = causes_grouped.copy()
    dff3 = grouped_df.copy()
    dff4 = causes_grouped2.copy()

    dff = wildfires_causes[
        (wildfires_causes['FIRE_YEAR'] >= year_range[0]) & (wildfires_causes['FIRE_YEAR'] <= year_range[1]) &
        (wildfires_causes['state_descriptions'].isin(selected_states))]

    dff2 = causes_grouped[
        (causes_grouped['FIRE_YEAR'] >= year_range[0]) & (causes_grouped['FIRE_YEAR'] <= year_range[1]) &
        (causes_grouped['state_descriptions'].isin(selected_states))]
    
    dff3 = grouped_df[
        (grouped_df['FIRE_YEAR'] >= year_range[0]) & (grouped_df['FIRE_YEAR'] <= year_range[1]) &
        (grouped_df['state_descriptions'].isin(selected_states))]

    dff4 = causes_grouped2[
        (causes_grouped2['FIRE_YEAR'] >= year_range[0]) & (causes_grouped2['FIRE_YEAR'] <= year_range[1]) &
        (causes_grouped2['state_descriptions'].isin(selected_states))]
    
    updated_chart1 = create_altair_chart(dff)
    updated_chart2 = create_altair_chart2(dff2)
    updated_chart3 = create_altair_chart3(dff3)
    updated_chart4 = create_altair_chart4(dff4)
    
    return updated_chart1, updated_chart2, updated_chart3, updated_chart4

if __name__ == '__main__':
    app.run_server(debug=True)